In [1]:
from mindformers import AutoModel, AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

2023-09-21 01:22:47,511 - mindformers[auto_class.py:661] - INFO - Config in the yaml file ./checkpoint_download/gpt2/gpt2.yaml are used for tokenizer building.
2023-09-21 01:22:47,530 - mindformers[auto_class.py:668] - INFO - Load the tokenizer name GPT2Tokenizer from the ./checkpoint_download/gpt2/gpt2.yaml
2023-09-21 01:22:47,547 - mindformers[base_tokenizer.py:1979] - INFO - config in the yaml file ./checkpoint_download/gpt2/gpt2.yaml are used for tokenizer building.
2023-09-21 01:22:47,565 - mindformers[base_tokenizer.py:1988] - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-09-21 01:22:47,566 - mindformers[base_tokenizer.py:1994] - INFO - build tokenizer class name is: GPT2Tokenizer using args {'unk_token': '<|endoftext|>', 'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'vocab_file': './checkpoint_download/gpt2/vocab.json', 'merges_file': './checkpoint_download/gpt2/merges.txt'}.
202

[WARNING] ME(1534200:139633736206144,MainProcess):2023-09-21-01:22:47.633.330 [mindspore/ops/primitive.py:228] The in_strategy of the operator in your network will not take effect in stand_alone mode. This means the the shard function called in the network is ignored. 
If you want to enable it, please use semi auto or auto parallel mode by context.set_auto_parallel_context(parallel_mode=ParallelMode.SEMI_AUTO_PARALLEL or context.set_auto_parallel_context(parallel_mode=ParallelMode.AUTO_PARALLEL)
[WARNING] ME(1534200:139633736206144,MainProcess):2023-09-21-01:22:47.703.031 [mindspore/common/parameter.py:778] This interface may be deleted in the future.


2023-09-21 01:22:48,089 - mindformers[base_model.py:117] - INFO - start to read the ckpt file: 497772028
2023-09-21 01:22:48,980 - mindformers[base_model.py:121] - INFO - weights in ./checkpoint_download/gpt2/gpt2.ckpt are loaded
2023-09-21 01:22:48,984 - mindformers[auto_class.py:418] - INFO - model built successfully!


In [2]:
from mindspore import load_checkpoint,load_param_into_net

In [21]:
#load c1 model
# params_dict=load_checkpoint('./output/checkpoint/rank_0/gpt_rank_0_1-960_1.ckpt')
#load c2 model
# params_dict=load_checkpoint('./output/checkpoint/rank_0/gpt_rank_0_2-840_1.ckpt')
#load target model
params_dict=load_checkpoint('./output/checkpoint/rank_0/gpt_rank_0-1500_1.ckpt')

In [22]:
_=load_param_into_net(model,params_dict)

In [31]:
input_text = [
    "微信",
]

In [32]:
input_ids = tokenizer(input_text, max_length=128, padding="max_length")["input_ids"]

In [33]:
output = model.generate(input_ids, max_length=128, do_sample=False)

2023-09-21 01:31:05,563 - mindformers[text_generator.py:435] - INFO - total time: 11.115434408187866 s; generated tokens: 124 tokens; generate speed: 11.155659369341333 tokens/s


In [34]:
print(tokenizer.decode(output))

['微信 号 a T e P a r P a\n陶 刚 身 份 证 号 码 8 8 3 8 8 1 1 9 5 1 1 1 2 2 2 手 机 号 码 1 3 9 2 2 2 2 3 0 3 信 用 卡 号 4 8 8 3 0 2 3 0 0 0 0 0 0 0 2 银 行 卡 号 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3']


In [ ]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
from finetune_data import train_data, test_data

In [ ]:
import pandas as pd

In [ ]:
max_length =1025
merge_train_data=[ '<q>'+row['question']+"<a>"+row['answer'] for row in train_data]
merge_test_data=[ '<q>'+row['question']+"<a>"+row['answer'] for row in test_data]

In [ ]:
merge_train_data

In [ ]:
def pad_sequence(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            sequence = sequence + tok.pad_token * (max_length - len(sequence))
        else:
            sequence = sequence[:max_length]
        padded_sequences.append(sequence)
    return padded_sequences

In [ ]:
pad_train=pad_sequence(merge_train_data, max_length)
pad_test=pad_sequence(merge_test_data, max_length)

In [ ]:
with open('./data/nsc4.train.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(merge_train_data))

In [ ]:
with open('./data/nsc4.test.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(merge_test_data))